In [3]:
!mv .kaggle /'content/kaggle.json'

mv: cannot stat '.kaggle': No such file or directory


In [4]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions list

ref                                                                deadline             category             reward  teamCount  userHasEntered  
-----------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                                                   2030-01-01 00:00:00  Getting Started   Knowledge       2314           False  
titanic                                                            2030-01-01 00:00:00  Getting Started   Knowledge      16615            True  
house-prices-advanced-regression-techniques                        2030-01-01 00:00:00  Getting Started   Knowledge       4624            True  
connectx                                                           2030-01-01 00:00:00  Getting Started   Knowledge        372           False  
competitive-data-science-predict-future-sales                      2020-12-31 23:59:00  Playground            Kudos       5923    

In [6]:
!kaggle datasets download -d wcukierski/enron-email-dataset

 99% 353M/358M [00:04<00:00, 92.3MB/s]
100% 358M/358M [00:04<00:00, 89.6MB/s]


In [7]:
!unzip '/content/enron-email-dataset.zip'

Archive:  /content/enron-email-dataset.zip
  inflating: emails.csv              


In [0]:
import pandas as pd
emails = pd.read_csv('emails.csv')
email_subset = emails[:10000]

In [17]:
email_subset['message'][1]

"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nTraveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what 

In [0]:
def parse_raw_message(raw_message):
  lines=raw_message.split('\n')
  email={}
  message=''
  for line in lines:
    if ':' not in line:
      message+=line.strip()
      emails['body']=message
      print(message)
    else:
      pairs=line.split(':')
      key=pairs[0].lower()
      value=pairs[1].strip()
      emails[key]=value
  return emails

In [0]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to','subject']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [0]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [0]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from'),
        'Subject': map_to_list(emails, 'Subject')
    }

In [28]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head())

                                                body  ... Subject
0                               Here is our forecast  ...        
1  Traveling to have a business meeting takes the...  ...        
2                     test successful.  way to go!!!  ...        
3  Randy,Can you send me a schedule of the salary...  ...        
4                                                     ...        

[5 rows x 4 columns]


In [0]:
email_df['body+subject']=email_df['body']+email_df['Subject']

In [31]:
email_df.head()

,body,to,from_,Subject,body+subject
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,,Here is our forecast
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,,Traveling to have a business meeting takes the...
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,,test successful. way to go!!!
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,,"Randy,Can you send me a schedule of the salary..."
4,,greg.piper@enron.com,phillip.allen@enron.com,,


In [32]:
import re
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
Lemmitizer=WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize 
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:

list1=[]
for i in range(0,len(email_df)):
  review=re.sub('[^a-zA-Z]',' ',email_df['body'][i])
  review=review.lower()
  review=word_tokenize(review)
  review = [w for w in review if not w in stop_words] 
   #review=review.split()
  pos_tag=nltk.pos_tag(review)
  list1.append(pos_tag)


# In[99]:


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'


# In[100]:


lemma=[]

for word in list1:
  #print(word)
  l=[]
  for p in word:
    #print(p)
    m=Lemmitizer.lemmatize(p[0], get_wordnet_pos(p[1]))
    #print(p, m)
    l.append(m)

  l = " ".join(l)
  lemma.append(l)

In [39]:
lemma

['forecast',
 'travel business meeting take fun trip especially prepare presentation would suggest hold business plan meeting take trip without formal business meeting would even try get honest opinion whether trip even desire necessary far business meeting think would productive try stimulate discussion across different group work often presenter speaks others quiet wait turn meeting might well hold round table discussion format suggestion go austin play golf rent ski boat jet ski fly somewhere take much time',
 'test successful way go',
 'randy send schedule salary level everyone thescheduling group plus thought change need make patti example phillip',
 '',
 'greg either next tuesday thursday phillip',
 'phillip allen pallen enron com mike grigsby mike grigsby enron com keith holst kholst enron com monique sanchezfrank ermisjohn lavoratothank helpphillip allen',
 '',
 'think require isp static ip address',
 'forward phillip k allen hou ect phillip discuss phone conversation parallon 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
f=tfidf.fit_transform(lemma)
features=pd.DataFrame(f.toarray(),columns=tfidf.get_feature_names())

In [0]:
from sklearn.decomposition import  TruncatedSVD,NMF

In [0]:
import numpy as np


In [40]:
model=NMF(n_components=4,random_state=46,alpha=0.1, l1_ratio=0.5, init="nndsvd")


H=model.fit_transform(f)



W=model.components_


def display_topics(H,W,feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])

display_topics(W,H,tfidf.get_feature_names(),lemma, 4,3)

Topic 0:
enron com mike corp
bob ambrocik enron com bridgette anderson enron com aaron armstrong enron com bryan aubuchon enron com c aucoin enron com ashraf ayyat enron com bilal bajwa enron com briant baker enron com arun balasundaram enron com andres balmaceda enron com angela barnett enron com andreas barschkis enron com amit bartarya enron com bryce baxter enron com antoinette beale enron com angeles beltri enron com aaron berutti enron com amy bundscho enron com bryan burch enron com aliza burgess enron com andrew burn enron com adam caldwell enron com anthony campos enron com brenda cassel enron com amy cavazos enron com betty chan enron com aneela charania enron com alejandra chavez enron com angela chen enron com benjamin chi enron com andy chun enron com r conner enron com audrey cook enron com barbara cook enron com brooklyn couch enron com beth cowan enron com bob crane enron com bryan critchfield enron com bernard dahanayake enron com andrea dahlke enron com brian davis en